##Checking if a SMILES code is available for a given CAS number
we must have a matrix with CAS number

In [ ]:
import pandas as pd
from pubchempy import get_compounds
data = pd.read_excel('CAS.xlsx')
cas_numbers = data['CAS'].tolist()

In [ ]:
def cas_to_smiles(cas_number):
    try:
        compound = get_compounds(cas_number, 'name')[0]
        smiles = compound.canonical_smiles
        return smiles
    except:
        return None

In [ ]:
data['SMILES'] = [cas_to_smiles(cas) for cas in cas_numbers]

In [ ]:
data.to_excel('wyniki_NAME.xlsx', index=False)

In [ ]:
data3 = pd.read_excel('PFAS_12_k.xlsx')
data3.merge(data, how='left', on='CAS')

Function to check the charge of a molecule and return SMILES

In [ ]:
from rdkit import Chem

In [ ]:
def check_formal_charge_and_smiles(smiles):
    smiles_str = str(smiles)
    molecule = Chem.MolFromSmiles(smiles_str)
    if molecule is None:
        return None, None
    total_charge = 0
    for atom in molecule.GetAtoms():
        atom_charge = atom.GetFormalCharge()
        total_charge += atom_charge
    return total_charge, smiles_str

In [ ]:
smiles = data['SMILES']

In [ ]:
# Create a new column of charges
data3['Ładunek'], data3['SMILES'] = zip(*smiles.apply(check_formal_charge_and_smiles))

In [ ]:
# Saving results
data3.to_excel('plik.xlsx', index=False)

##Rename files (if they were deleted during structure generation)

We need a matrix with columns "Old name", "New name" and "Charge", which is generated from the previous code

In [ ]:
import os
import pandas as pd
import fileinput

def rename_and_modify_mop_files(df, directory_path):
    for index, row in df.iterrows():
        old_filename = str(row["Old Filename"])
        new_filename = str(row["New Filename"])
        charge = str(row["Charge"])

        # Add the ".mop" extension if not present in the new file name
        if not new_filename.endswith(".mop"):
            new_filename += ".mop"

        old_file_path = os.path.join(directory_path, old_filename + ".mop")
        new_file_path = os.path.join(directory_path, new_filename)

        try:
            # Rename the file
            os.rename(old_file_path, new_file_path)

            # Modify the first line in the new file
            with fileinput.FileInput(new_file_path, inplace=True, backup='.bak') as file:
                for line in file:
                    # Replace the line starting with "PUT KEYWORDS HERE" with the desired keywords
                    if line.startswith("PUT KEYWORDS HERE"):
                        line = f"PM7 PRECISE PDBOUT CHARGE={charge} CYCLES=99999999\n"
                    print(line, end='')

            print(f"Renamed and modified file: {old_filename}.mop -> {new_filename}")

        except FileNotFoundError:
            print(f"File {old_filename}.mop does not exist. Skipping.")

# Set the correct directory path
directory_path = '/Users/michalina/Desktop/struktury_12k/nieruszone mopy-kopia'

# Read the Excel file with old and new file names
df = pd.read_excel("nazwy_plikow1.xlsx")

# Rename the files in the directory and modify the keywords based on data in the Excel sheet
rename_and_modify_mop_files(df, directory_path)